### 10.3.3 수식 전개하기
#### 수식의 전개중 임의의 합을 거듭제곱하거나 다른 수식과 곱해서 수행하는 것을 진행하겠다.

In [1]:
from abc import ABC, abstractmethod
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass

In [2]:
# 식이나 변수나 수라면 이미 전개가 끝났다. 이런 경우에는 expand 메서드는 객체 자제를 리턴한다.
class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1 = self.exp1.expand() # 곱의 두 항을 전개한다.
        expanded2 = self.exp2.expand()
        if isinstance(expanded1, Sum): # 곱의 첫번쨰 항이 sum이면 첫번쨰 항의 각 항과 곱의 두번째 항을 곱하는 Product를 구한뒤
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps]) # 그 전개(expand)메서드를 호출한다. 곱의 두번째 항이 Sum일 수 있기 때문이다.
        elif isinstance(expanded2, Sum): # 곱의 두번째 항이 Sum이면 곱의 첫번째 항에다가 두번째 항의 각 항을 곱한다.
            return Sum(*[Product(expanded1,e) for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2) # 두 항이 모두 sum이 아니면 분배법칙을 호출하지 않는다.
    def display(self):
        return "Product({},{})".format(self.exp1.display(),self.exp2.display())
        

In [6]:
class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self,**bindings):
        try:
            return bindings[self.symbol]
        except:
            raise KeyError("Variable '{}' is not bound.".format(self.symbol))
            
class Sum():
    def __init__(self,*exps): # 입력하는 항의 개수에 제한이 없음
        self.exps = exps

In [7]:
Y = Variable('y')
Z = Variable('z')
A = Variable('a')
B = Variable('b')

Product(Sum(A,B),Sum(Y,Z))